## Setup

In [1]:
# Setup
import geopandas as gpd
import flopy
import numpy as np
import os
import pandas as pd

In [2]:
cwd = '\\'.join(os.getcwd().split('\\')[:-2])
cwd

'c:\\repos\\hm\\MAURICE'

## Tests

### Test: Load the model

In [ ]:
# bug detected:
# 1. if the script is not in the same folder as the .nam file,
#    an error arises saying that the ._kx file is not found
# 2. if the .sfr file is in a SFR2 format (e.g. NSRTM = 2),
#    the file doesn't get read

m = flopy.mfusg.MfUsg.load(os.path.join(os.getcwd(), 't_m11_C1.nam'))

OSError: cannot find name file: c:\repos\hm\MAURICE\test_models\m11_C1\tool_m11_c1\t_m11_C1.nam

In [18]:
# m.sfr.reach_data

In [33]:
# m.sfr.segment_data

In [ ]:
sp = 1 #stress period number
m.sfr.segment_data[sp-1]

In [12]:
m.sfr.nstrm

np.int64(174)

In [14]:
m.sfr.isfropt

0

In [ ]:
m.sfr.isfropt = 2
#setting it from here doesn't change the output format

#### Test: load sfr condition from shapefile

In [4]:
sfr_shp = gpd.read_file(os.path.join(cwd, 'test_models', 'SFR_ICALC2.shp'))

In [6]:
sfr_shp.shape

(174, 254)

In [8]:
sfr_shp.Slope

0      0.000019
1      0.000019
2      0.000019
3      0.000019
4      0.000019
         ...   
169    0.000019
170    0.000019
171    0.000019
172    0.000019
173    0.000019
Name: Slope, Length: 174, dtype: float64

### Test: Modify the segment data parameters

In [3]:
# Load the data to be set up into the different stress periods
q = pd.read_csv(os.path.join(cwd, 'test_models', 'portate_segmenti.csv'))
q[q.isna()] = 0

In [ ]:
for sp in range(1, 91+1):
    tool = q.loc[q.SP == sp, ['Qin', 'Sg_2', 'Sg_3', 'Sg_4', 'Sg_5', 'Sg_6', 'Sg_7']].values[0]
    tool = np.around(tool, 5)
    tool = [tool[0]] + [-x if x != 0 else 0 for x in tool[1:]]
    m.sfr.segment_data[sp-1].flow = tool

### Test: write the new .sfr file

In [16]:
m.model_ws = os.path.join(cwd, 'test_models', 'tool_m11_c1', 'test_datasets_m11_c1')

In [19]:
m.write_input()

## Test: load reach and segment data from Excel file

In [13]:
sfr_data = os.path.join(cwd, 'test_models', 'm11_C1_sfr_data.xlsx')

# Load general parameters (item 1)
it1 = pd.read_excel(sfr_data, sheet_name = 'ITEM1')

# Load reach data (item 2)
reach_data = pd.read_excel(sfr_data, sheet_name = 'ITEM2b')
reach_data = reach_data.apply(pd.to_numeric)
reach_data.columns = ['nreach', 'iseg', 'ireach', 'rchlen', 'strtop', 'slope',  'strthick',  'strhc1']
reach_data = reach_data.loc[:,:].to_records(index = False)
# flopy adds 1 to node???
# reach_data.node = reach_data.node - 1

# Load item 5
it5 = pd.read_excel(sfr_data, sheet_name = 'ITEM5')

# Load segment data (item 6a)
segment_data = pd.read_excel(sfr_data, sheet_name = 'ITEM6abc')
segment_data.columns = [x.lower() for x in segment_data.columns]
segment_data = segment_data.loc[:,:].to_records(index = False)
segment_data = {0: segment_data}

In [14]:
reach_data

rec.array([(4553, 1,  1, 106, 178.594126, 0.00063, 0.5, 1.e-06),
           (4554, 1,  2,   8, 178.592659, 0.00063, 0.5, 1.e-06),
           (4677, 1,  3,  99, 178.582724, 0.00063, 0.5, 1.e-06),
           (4678, 1,  4,  63, 178.568748, 0.00063, 0.5, 1.e-06),
           (4679, 2,  1,  99, 178.553359, 0.00063, 0.5, 1.e-06),
           (4680, 2,  2,  97, 178.548897, 0.00063, 0.5, 1.e-06),
           (4803, 2,  3,   4, 178.543434, 0.00063, 0.5, 1.e-06),
           (4804, 2,  4, 100, 178.537512, 0.00063, 0.5, 1.e-06),
           (4805, 2,  5, 106, 178.527313, 0.00063, 0.5, 1.e-06),
           (4806, 2,  6,  88, 178.518107, 0.00063, 0.5, 1.e-06),
           (4929, 2,  7,  32, 178.512193, 0.00063, 0.5, 1.e-06),
           (4930, 2,  8, 122, 178.504409, 0.00063, 0.5, 1.e-06),
           (5053, 2,  9,   9, 178.498191, 0.00063, 0.5, 1.e-06),
           (5054, 2, 10, 106, 178.493187, 0.00063, 0.5, 1.e-06),
           (5055, 3,  1,  21, 178.472727, 0.00063, 0.5, 1.e-06),
           (5178, 3,  2, 

In [15]:
m = flopy.modflow.Modflow()
sfr = flopy.modflow.ModflowSfr2(
    m,
    nstrm = it1.NSTRM.values[0],              # number of reaches
    nss = it1.NSS.values[0],                  # number of segments
    const = it1.CONST.values[0],              # constant for manning's equation: 1 for m/s
    dleak = it1.DLEAK.values[0],              # closure tolerance for stream stage computation
    ipakcb = it1.ISTCB1.values[0],            # flag for writing SFR output to cell-by-cell budget (on unit 50)
    istcb2 = it1.ISTCB2.values[0],            # flag for writing SFR output to text file
    dataset_5 = {0: it5.values[0].tolist()},
    unit_number = unit_number,
    isfropt = it1.ISFROPT.values[0],          # ISFROPT parameter, defines input file format writing
    reach_data = reach_data,
    segment_data = segment_data
)

NameError: name 'unit_number' is not defined